<a href="https://colab.research.google.com/github/VSennaa/text-classification-bigdata-agnews-spark/blob/main/text_classification_bigdata_agnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/datasets/sh0416/ag_news

In [1]:
from pyspark.sql import SparkSession
import os

Inicizalizando o Spark


In [2]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Aquisição do dataset

In [3]:
!wget 'https://raw.githubusercontent.com/VSennaa/text-classification-bigdata-agnews-spark/refs/heads/main/train.jsonl'
!wget 'https://raw.githubusercontent.com/VSennaa/text-classification-bigdata-agnews-spark/refs/heads/main/test.jsonl'

--2025-08-05 21:03:15--  https://raw.githubusercontent.com/VSennaa/text-classification-bigdata-agnews-spark/refs/heads/main/train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33697315 (32M) [text/plain]
Saving to: ‘train.jsonl’

train.jsonl         100%[===================>]  32.14M   177MB/s    in 0.2s    

2025-08-05 21:03:15 (177 MB/s) - ‘train.jsonl’ saved [33697315/33697315]

--2025-08-05 21:03:15--  https://raw.githubusercontent.com/VSennaa/text-classification-bigdata-agnews-spark/refs/heads/main/test.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request se

In [4]:
df_train = spark.read.json("/content/train.jsonl")
df_test = spark.read.json("/content/test.jsonl")